In [ ]:
import numpy as np
import pandas as pd
import random
import time
import math
import os
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter as sg_filter
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from dateutil import parser
from pathlib import Path
import json 
import shutil

from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import correct_predictor_columns, correct_timestamps, rolling_stats, input_data_split
from wattile.time_processing import add_processed_time_columns
from wattile.models.liangs_model import main as liangs_model
PROJECT_DIRECTORY = Path().resolve().parent.parent

### reading configs

In [ ]:
"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
##################################################################################
# choose the configs file to use as an input
##################################################################################
# # main configs file
# with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
#     configs = json.load(f)
##################################################################################
# code testing configs file
with open(PROJECT_DIRECTORY / "tests" / "fixtures" / "test_configs.json", "r") as f:
    configs = json.load(f)
##################################################################################

exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"
if exp_dir.exists():
    shutil.rmtree(exp_dir)
exp_dir.mkdir()

configs["exp_dir"] = str(exp_dir)
configs["num_epochs"] = 10
configs["data_dir"] = str(PROJECT_DIRECTORY / "tests" / "data")

configs

### override config if necessary

In [ ]:
# configs["feat_stats"]["window_width"] = '15min'
# configs["feat_stats"]["window_increment"] = '15min'

### reading data

In [ ]:
site = configs['building']

In [ ]:
data = read_dataset_from_file(configs)
data

### feature extraction

In [ ]:
# assert we have the correct columns and order them
data = correct_predictor_columns(configs, data)

# sort and trim data specified time period
data = correct_timestamps(configs, data)

# Add time-based features
data = add_processed_time_columns(data, configs)

# Add statistics features
data = rolling_stats(data, configs)

data

In [ ]:
for loss_function_qs in [0.05, 0.25, 0.50, 0.75, 0.95]: #[0.50]:#[0.05, 0.50, 0.95]:
    print(f"Processing loss function alpha: {loss_function_qs}")
    s, s2, mape_s, mae_s, mae, mape, total_mins, train_loss, test_loss, for_plotting, MU, STD = liangs_model(
        data=data,
        configs=configs
    )

### visualization

In [ ]:
q95 = pd.read_csv(f'results/{site}/q0.95.csv')
q75 = pd.read_csv(f'results/{site}/q0.75.csv')
q50 = pd.read_csv(f'results/{site}/q0.5.csv')
q25 = pd.read_csv(f'results/{site}/q0.25.csv')
q05 = pd.read_csv(f'results/{site}/q0.05.csv')
actual = pd.read_csv(f'results/{site}/actual.csv')

In [ ]:
plt.figure(figsize=(15, 4.5))

# calculating error
y_actual = actual
y_predicted = q50.values.flatten()
MSE = mean_squared_error(y_actual, y_predicted)
RMSE = MSE**0.5
CV_RMSE = RMSE/y_actual.mean().values[0]
mae = mean_absolute_error(y_actual, y_predicted)

plt.title(f'{site} Whole Building Real Power Total Prediction\nError: MAE:{mae:.2f} kW, CV-RMSE:{CV_RMSE*100:.2f}%')

plt.plot(actual, label = 'actual')
plt.plot(sg_filter(q95.values.flatten(),5,2), label = 'Q95', color = '#a2a2a2')
plt.plot(sg_filter(q75.values.flatten(),5,2), label = 'Q75', color = '#5b5b5b')
plt.plot(sg_filter(q50.values.flatten(),5,2), label = 'Medium', color = '#322b2b')
plt.plot(sg_filter(q25.values.flatten(),5,2), label = 'Q25', color = '#5b5b5b')
plt.plot(sg_filter(q05.values.flatten(),5,2), label = 'Q5', color = '#a2a2a2')


plt.legend()

# plt.ylim(20,40)
# n=0
# plt.xlim(1000, 1720)

plt.ylabel(f'{site} Whole Building Real Power Total')
plt.xlabel('Timesteps, 1 min/unit, showing 33 hours results')

plt.show()